## Imports and initial look at data

In [ ]:
from pprint import pprint
import pandas as pd
import os
import jsonlines
pd.set_option('display.max_columns', None)
import json
import warnings
warnings.filterwarnings('ignore')


## Extract JSONL into dataframes
Separate df for product page, reviews page, and bestsellers page

In [ ]:
jsonl_files = [
    'dataset_Amazon-crawler_2023-06-16_09-05-55-623.jsonl', 
    'dataset_Amazon-crawler_2023-06-16_10-33-19-259.jsonl', 
    'dataset_Amazon-crawler_2023-06-16_10-22-09-925.jsonl', 
    'dataset_amazon-bestsellers_2023-06-16_13-09-24-209.jsonl', 
    'dataset_amazon-bestsellers_2023-06-16_13-03-46-148.jsonl', 
    'dataset_amazon-reviews-scraper_2023-06-16_12-41-00-735.jsonl', 
    'dataset_amazon-reviews-scraper_2023-06-16_09-35-54-585.jsonl', 
    'dataset_amazon-reviews-scraper-task_2023-06-16_11-03-38-627.jsonl'
]

crawler_data = []
bestsellers_data = []
reviews_data = []

for jsonl_file in jsonl_files:
    if jsonl_file.endswith('.jsonl'):
        
        with open(f'/amazon_data/{jsonl_file}') as file:
            print(jsonl_file, len(file.readlines()))
        with open(f'/amazon_data/{jsonl_file}') as file:
            if 'crawler' in jsonl_file:
                for line in file:
                    crawler_data.append(json.loads(line))
            elif 'bestsellers' in jsonl_file:
                for line in file:
                    bestsellers_data.append(json.loads(line))
            elif 'reviews' in jsonl_file:
                for line in file:
                    reviews_data.append(json.loads(line))

crawler_df = pd.DataFrame(crawler_data)
bestsellers_df = pd.DataFrame(bestsellers_data)
reviews_df = pd.DataFrame(reviews_data)

### Get initial info on raw JSONL data

In [ ]:
for dftype, df in zip(['crawler', 'bestsellers', 'reviews'],
              [crawler_df, bestsellers_df, reviews_df]):
    print(dftype)
    print(f'shape: {df.shape}')
    print(f'firstcol: {df.columns[0]}')
    print(f'lastcol: {df.columns[-1]}')
    
    null_cols = df.columns[df.isnull().all()].tolist()
    print(f'null_cols: {null_cols} (dropping)')
    df.drop(null_cols, axis=1, inplace=True)

    null_counts = df.isnull().sum()
    null_counts_df = pd.DataFrame(null_counts, columns=['Null Count'])
    null_counts_df['Null Percent'] = null_counts_df['Null Count'] / df.shape[0]
    print('Cols with nulls:')
    display(null_counts_df[null_counts_df['Null Count'] > 0])

    list_cols = [col for col in df.columns if isinstance(df[col].iloc[0], list) or isinstance(df[col].iloc[0], dict)]
    print(f'cols containing dict or list: {list_cols}')

    print(f'{dftype}_df:')
    display(df.head(3))


    df.to_excel(f'/amazon_data/{dftype}_agg_df.xlsx')

crawler
shape: (2157, 32)
firstcol: title
lastcol: locationText
null_cols: ['shippingPrice', 'priceVariants'] (dropping)
Cols with nulls:


,Null Count,Null Percent
price,87,0.040334
listPrice,1326,0.614743
stars,252,0.116829
reviewsCount,252,0.116829
answeredQuestions,626,0.290218
description,705,0.326843
reviewsLink,353,0.163653
delivery,97,0.044970
fastestDelivery,483,0.223922
returnPolicy,2147,0.995364


cols containing dict or list: ['price', 'listPrice', 'starsBreakdown', 'features', 'attributes', 'variantAsins', 'variantAttributes', 'manufacturerAttributes', 'seller']
crawler_df:


,title,url,asin,inStock,inStockText,brand,price,listPrice,stars,starsBreakdown,reviewsCount,answeredQuestions,breadCrumbs,thumbnailImage,description,features,attributes,variantAsins,reviewsLink,hasReviews,delivery,fastestDelivery,returnPolicy,support,variantAttributes,manufacturerAttributes,seller,bestsellerRanks,saleSummary,locationText
0,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",https://www.amazon.com/dp/B0BNM4WLBJ,B0BNM4WLBJ,True,Only 19 left in stock - order soon,sinbun,"{'value': 529, 'currency': '$'}","{'value': 579, 'currency': '$'}",4.4,"{'5star': 0.7, '4star': 0.18, '3star': 0, '2st...",14.0,11.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/61oop4yoyF...,None,[【11th Intel Core i7 processor】11th i7 mini PC...,"[{'key': 'Processor', 'value': '4.7 GHz core_i...","[B0BNM4WLBJ, B0B8D2D7RN, B0BQ2BBRDV]",/i7-1165G7-Thunderbolt-Desktop-Computer-Comput...,True,"Wednesday, May 17","Tomorrow, May 12",None,None,"[{'key': 'Size', 'value': 'I7-1165G7 16G/512G'}]","[{'key': 'Thunderbolt 4.0 Type-C Port', 'value...","{'name': 'Sinbun Direct', 'id': 'A26EVMAO82NEO...",None,Save $50.00 with coupon,Select your address
1,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",https://www.amazon.com/dp/B0BQVM2XL6,B0BQVM2XL6,True,In Stock In Stock.,GEEKOM,"{'value': 599.99, 'currency': '$'}","{'value': 749.99, 'currency': '$'}",4.7,"{'5star': 0.8, '4star': 0.15, '3star': 0, '2st...",54.0,35.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/51W6MBfVCT...,None,[[A Mini PC with An Upgraded Processor] GEEKOM...,"[{'key': 'Standing screen display size', 'valu...","[B0BQVM2XL6, B0BQWCQ7ZY]",/GEEKOM-Mini-IT11-i7-11390H-Computer/product-r...,True,"Wednesday, May 17","Tomorrow, May 12",None,None,"[{'key': 'Size', 'value': 'i711390H-32GB+1TB'}]",[],"{'name': 'GEEKOM Brand Store', 'id': 'A3FHF27Z...",None,None,Select your address
2,Dell Precision Tower 3420 SFF Intel Core i7-77...,https://www.amazon.com/dp/B0958ZM66D,B0958ZM66D,True,In Stock.,Amazon Renewed,"{'value': 533.99, 'currency': '$'}",None,NaN,"{'5star': 0, '4star': 0, '3star': 0, '2star': ...",NaN,NaN,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/616n0bPSxb...,Boost up your computing experience with Refurb...,[This Certified Refurbished product is tested ...,"[{'key': 'Processor', 'value': '3.6 GHz core_i...",[],NaN,False,May 18 - 25,May 15 - 18,None,None,[],[],"{'name': 'Joy Systems Inc', 'id': 'APEGUV39VCX...",None,None,Select your address


bestsellers
shape: (200, 37)
firstcol: position
lastcol: locationText
null_cols: ['shippingPrice', 'priceVariants', 'bestsellerRanks'] (dropping)
Cols with nulls:


...

,position,category,categoryUrl,name,price,currency,numberOfOffers,url,thumbnail,title,asin,inStock,inStockText,brand,listPrice,stars,starsBreakdown,reviewsCount,answeredQuestions,breadCrumbs,thumbnailImage,description,features,attributes,variantAsins,reviewsLink,delivery,fastestDelivery,returnPolicy,support,variantAttributes,manufacturerAttributes,seller,locationText
0,30,Amazon Best Sellers: Best Desktop Computers,https://www.amazon.com/Best-Sellers-Computers-...,"Beelink Mini PC, Intel 12th Gen Alder Lake- N9...",159.00,$,1,https://www.amazon.com/dp/B0BVLPCDVW,https://images-na.ssl-images-amazon.com/images...,"Beelink Mini PC, Intel 12th Gen Alder Lake- N9...",B0BVLPCDVW,True,In Stock,Beelink,"{'value': 199, 'currency': '$'}",4.6,"{'5star': 0.79, '4star': 0.14, '3star': 0.03, ...",115.0,19.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/71yonCCmlv...,None,[✔️12th Intel Alder Lake-N95 - Beelink Mini S1...,"[{'key': 'Processor', 'value': '3.4 GHz none'}...","[B0BVLPCDVW, B0BVLS7ZHP, B09ZLBDVZD, B09ZLDBC8X]",/Beelink-Desktop-Computer-Support-Ethernet/pro...,"Wednesday, May 17","Tomorrow, May 12",None,None,"[{'key': 'Size', 'value': 'N95 8G+256G'}]",[],"{'name': 'Beelink US', 'id': 'A2RITUENS753LH',...",Select your address
1,27,Amazon Best Sellers: Best Desktop Computers,https://www.amazon.com/Best-Sellers-Computers-...,"Mini PC Windows 11 Pro, KAMRUI AK1 PRO 12GB RA...",199.99,$,1,https://www.amazon.com/dp/B099KSJN8P,https://images-na.ssl-images-amazon.com/images...,"Mini PC Windows 11 Pro, KAMRUI AK1 PRO 12GB RA...",B099KSJN8P,True,In Stock In Stock.,KAMRUI,"{'value': 259.99, 'currency': '$'}",4.4,"{'5star': 0.68, '4star': 0.17, '3star': 0.05, ...",876.0,156.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/71LgGofvm1...,None,[[Small but Powerful]-Mini PC about 5.04 x 5.0...,"[{'key': 'Standing screen display size', 'valu...",[],/Computer-Celeron-J3455-Processor-2-5-inch/pro...,"Wednesday, May 17","Tomorrow, May 12",None,None,[],[],"{'name': 'KAMRUI-US', 'id': 'A3R3T09IX9ERXX', ...",Select your address
2,29,Amazon Best Sellers: Best Desktop Computers,https://www.amazon.com/Best-Sellers-Computers-...,"HP 2022 Newest All-in-One Desktop, 21.5"" FHD D...",589.00,$,1,https://www.amazon.com/dp/B0B6523VBD,https://images-na.ssl-images-amazon.com/images...,"HP 2022 Newest All-in-One Desktop, 21.5"" FHD D...",B0B6523VBD,True,Only 3 left in stock - order soon,HP,None,4.4,"{'5star': 0.69, '4star': 0.17, '3star': 0.04, ...",174.0,28.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/91-CdWQGWA...,None,[【High Speed RAM And Enormous Space】16GB DDR4 ...,"[{'key': 'Standing screen display size', 'valu...",[],/HP-Desktop-Display-Processor-Keyboard/product...,May 15 - 17,None,None,None,[],[],"{'name': 'Hott Tech Geek', 'id': 'A3TYVCQOBOEC...",Select your address


reviews
shape: (23950, 14)
firstcol: productAsin
lastcol: position
null_cols: [] (dropping)
Cols with nulls:


,Null Count,Null Percent
reviewUrl,543,0.022672
countryCode,11874,0.495783


cols containing dict or list: ['reviewImages']
reviews_df:


,productAsin,ratingScore,reviewTitle,reviewUrl,reviewReaction,reviewedIn,date,country,countryCode,reviewDescription,isVerified,variant,reviewImages,position
0,B0BYJWRSC1,4,Painful Laptop,https://www.amazon.com/gp/customer-reviews/R6N...,,"Reviewed in the United States 🇺🇸 on May 6, 2023",2023-05-06,United States,🇺🇸,This is a fast and elegant looking Windows 11 ...,True,Capacity: 64GB RAM|2TB SSDStyle: Win11Pro,[],1
1,B0BYJWRSC1,5,Has all the features,https://www.amazon.com/gp/customer-reviews/R39...,,"Reviewed in the United States 🇺🇸 on April 20, ...",2023-04-20,United States,🇺🇸,This laptop is fast! 64 mb of ram really crank...,True,Capacity: 64GB RAM|2TB SSDStyle: Win11Pro,[],2
2,B0BYJWRSC1,5,Great Service,https://www.amazon.com/gp/customer-reviews/R3V...,,"Reviewed in the United States 🇺🇸 on April 26, ...",2023-04-26,United States,🇺🇸,Fast delivery and product exactly as described,True,Capacity: 64GB RAM|2TB SSDStyle: Win11Pro,[],3


## Data Cleaning

### Drop irrelevant columns

In [ ]:
crawler_df.drop(columns=[
                         'inStock',
                         'inStockText',
                         'listPrice',
                         'starsBreakdown',
                         'answeredQuestions',
                         'delivery',
                         'fastestDelivery',
                         'returnPolicy',
                         'support',
                         'seller',
                         'bestsellerRanks',
                         'saleSummary',
                         'locationText'],
                inplace=True)
crawler_df.drop_duplicates(subset=['asin'], keep='first', inplace=True) 

In [ ]:
bestsellers_df.drop(columns=[
                            'inStock',
                            'inStockText',
                            'listPrice',
                            'starsBreakdown',
                            'answeredQuestions',
                            'delivery',
                            'fastestDelivery',
                            'returnPolicy',
                            'support',
                            'seller',
                            'locationText'],
                    inplace=True)

In [ ]:
reviews_df.drop(columns=[
    'isVerified',
    'variant',
    'reviewImages',
    'position'],
    inplace=True)

### Remove miscategorized items and map AMZN section string to Category & Sub-Category fields

In [ ]:
crawler_df = crawler_df[~crawler_df['breadCrumbs'].str.contains('Camera')]
category_map = {         'Electronics › Computers & Accessories › Computers & Tablets › Desktops › Minis' : 'Desktops',
                     'Electronics › Computers & Accessories › Computers & Tablets › Desktops › Towers' : 'Desktops',
                     'Electronics › Computers & Accessories › Computers & Tablets › Desktops › All-in-Ones' : 'Desktops',
                     'Electronics › Computers & Accessories › Computers & Tablets › Laptops › Traditional Laptops' : 'Laptops',
                     'Electronics › Computers & Accessories › Computers & Tablets › Laptops › 2 in 1 Laptops' : 'Laptops',
                     'Electronics › Computers & Accessories › Computers & Tablets › Laptops' : 'Laptops'}

subcategory_map = {         'Electronics › Computers & Accessories › Computers & Tablets › Desktops › Minis' : 'Minis',
                     'Electronics › Computers & Accessories › Computers & Tablets › Desktops › Towers' : 'Towers',
                     'Electronics › Computers & Accessories › Computers & Tablets › Desktops › All-in-Ones' : 'All-in-Ones',
                     'Electronics › Computers & Accessories › Computers & Tablets › Laptops › Traditional Laptops' : 'Traditional',
                     'Electronics › Computers & Accessories › Computers & Tablets › Laptops › 2 in 1 Laptops' : '2-in-1',
                     'Electronics › Computers & Accessories › Computers & Tablets › Laptops' : 'Traditional'}

crawler_df['category'] = crawler_df['breadCrumbs'].map(category_map)
crawler_df['sub-category'] = crawler_df['breadCrumbs'].map(subcategory_map)

crawler_df.head()

,title,url,asin,brand,price,stars,reviewsCount,breadCrumbs,thumbnailImage,description,features,attributes,variantAsins,reviewsLink,hasReviews,variantAttributes,manufacturerAttributes,category,sub-category
0,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",https://www.amazon.com/dp/B0BNM4WLBJ,B0BNM4WLBJ,sinbun,"{'value': 529, 'currency': '$'}",4.4,14.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/61oop4yoyF...,None,[【11th Intel Core i7 processor】11th i7 mini PC...,"[{'key': 'Processor', 'value': '4.7 GHz core_i...","[B0BNM4WLBJ, B0B8D2D7RN, B0BQ2BBRDV]",/i7-1165G7-Thunderbolt-Desktop-Computer-Comput...,True,"[{'key': 'Size', 'value': 'I7-1165G7 16G/512G'}]","[{'key': 'Thunderbolt 4.0 Type-C Port', 'value...",Desktops,Minis
1,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",https://www.amazon.com/dp/B0BQVM2XL6,B0BQVM2XL6,GEEKOM,"{'value': 599.99, 'currency': '$'}",4.7,54.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/51W6MBfVCT...,None,[[A Mini PC with An Upgraded Processor] GEEKOM...,"[{'key': 'Standing screen display size', 'valu...","[B0BQVM2XL6, B0BQWCQ7ZY]",/GEEKOM-Mini-IT11-i7-11390H-Computer/product-r...,True,"[{'key': 'Size', 'value': 'i711390H-32GB+1TB'}]",[],Desktops,Minis
2,Dell Precision Tower 3420 SFF Intel Core i7-77...,https://www.amazon.com/dp/B0958ZM66D,B0958ZM66D,Amazon Renewed,"{'value': 533.99, 'currency': '$'}",NaN,NaN,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/616n0bPSxb...,Boost up your computing experience with Refurb...,[This Certified Refurbished product is tested ...,"[{'key': 'Processor', 'value': '3.6 GHz core_i...",[],NaN,False,[],[],Desktops,Towers
3,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,https://www.amazon.com/dp/B099CHVFJJ,B099CHVFJJ,AVGPC,"{'value': 549, 'currency': '$'}",3.9,23.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/61LAnQI-iH...,Product images used on this page are for illus...,[AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD ...,"[{'key': 'Processor', 'value': '4.8 GHz ryzen_...","[B099CHVFJJ, B09X2JCX8Z, B0BGYRWRH7, B099CSM7L...",/AVGPC-Q-Box-Gaming-PC-Q-Box_7/product-reviews...,True,"[{'key': 'Style', 'value': 'Mini_B_5700G'}]",[],Desktops,Towers
4,Apple 2023 Mac Mini Desktop Computer M2 chip w...,https://www.amazon.com/dp/B0BSHGHGXR,B0BSHGHGXR,Apple,"{'value': 559, 'currency': '$'}",4.6,184.0,,https://m.media-amazon.com/images/I/61La8PAa42...,None,[SUPERCHARGED BY M2 — Get more done faster wit...,[],"[B0BSHGHGXR, B0BSHFS6XX, B0BSHGHGXR, B0BSHGTPMV]",/Apple-Desktop-Computer-10%E2%80%91core-Ethern...,True,"[{'key': 'Style', 'value': 'Apple M2 Chip'}, {...",[],NaN,NaN


# Fill in missing Category data for Mac products
(for some reason 'breadCrumbs' and 'attributes' were not populated for some Apple products)

In [ ]:
# where breadCrumbs is null and 'brand'=='Apple', set 'category' to Desktops the string 'desktop' or 'mini' appears in the title, otherwise set it to Laptops
crawler_df.loc[(crawler_df['category'].isnull()) & (crawler_df['brand']=='Apple') & (crawler_df['title'].str.lower().str.contains('desktop') | crawler_df['title'].str.lower().str.contains('mini')), 'category'] = 'Desktops'
crawler_df.loc[(crawler_df['sub-category'].isnull()) & (crawler_df['brand']=='Apple') & (crawler_df['title'].str.lower().str.contains('desktop') | crawler_df['title'].str.lower().str.contains('mini')), 'sub-category'] = 'Minis'
crawler_df.loc[(crawler_df['category'].isnull()) & (crawler_df['title'].str.lower().str.contains('laptop')  | crawler_df['title'].str.lower().str.contains('macbook')), 'category'] = 'Laptops'
crawler_df.loc[(crawler_df['sub-category'].isnull()) & (crawler_df['title'].str.lower().str.contains('laptop')  | crawler_df['title'].str.lower().str.contains('macbook')), 'sub-category'] = 'Traditional'

# drop breadcrumbs -- extracted to category
crawler_df.drop(columns=['breadCrumbs'], inplace=True)

crawler_df.head()

,title,url,asin,brand,price,stars,reviewsCount,breadCrumbs,thumbnailImage,description,features,attributes,variantAsins,reviewsLink,hasReviews,variantAttributes,manufacturerAttributes,category,sub-category
0,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",https://www.amazon.com/dp/B0BNM4WLBJ,B0BNM4WLBJ,sinbun,"{'value': 529, 'currency': '$'}",4.4,14.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/61oop4yoyF...,None,[【11th Intel Core i7 processor】11th i7 mini PC...,"[{'key': 'Processor', 'value': '4.7 GHz core_i...","[B0BNM4WLBJ, B0B8D2D7RN, B0BQ2BBRDV]",/i7-1165G7-Thunderbolt-Desktop-Computer-Comput...,True,"[{'key': 'Size', 'value': 'I7-1165G7 16G/512G'}]","[{'key': 'Thunderbolt 4.0 Type-C Port', 'value...",Desktops,Minis
1,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",https://www.amazon.com/dp/B0BQVM2XL6,B0BQVM2XL6,GEEKOM,"{'value': 599.99, 'currency': '$'}",4.7,54.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/51W6MBfVCT...,None,[[A Mini PC with An Upgraded Processor] GEEKOM...,"[{'key': 'Standing screen display size', 'valu...","[B0BQVM2XL6, B0BQWCQ7ZY]",/GEEKOM-Mini-IT11-i7-11390H-Computer/product-r...,True,"[{'key': 'Size', 'value': 'i711390H-32GB+1TB'}]",[],Desktops,Minis
3,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,https://www.amazon.com/dp/B099CHVFJJ,B099CHVFJJ,AVGPC,"{'value': 549, 'currency': '$'}",3.9,23.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/61LAnQI-iH...,Product images used on this page are for illus...,[AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD ...,"[{'key': 'Processor', 'value': '4.8 GHz ryzen_...","[B099CHVFJJ, B09X2JCX8Z, B0BGYRWRH7, B099CSM7L...",/AVGPC-Q-Box-Gaming-PC-Q-Box_7/product-reviews...,True,"[{'key': 'Style', 'value': 'Mini_B_5700G'}]",[],Desktops,Towers
4,Apple 2023 Mac Mini Desktop Computer M2 chip w...,https://www.amazon.com/dp/B0BSHGHGXR,B0BSHGHGXR,Apple,"{'value': 559, 'currency': '$'}",4.6,184.0,,https://m.media-amazon.com/images/I/61La8PAa42...,None,[SUPERCHARGED BY M2 — Get more done faster wit...,[],"[B0BSHGHGXR, B0BSHFS6XX, B0BSHGHGXR, B0BSHGTPMV]",/Apple-Desktop-Computer-10%E2%80%91core-Ethern...,True,"[{'key': 'Style', 'value': 'Apple M2 Chip'}, {...",[],Desktops,Minis
6,"HP Chromebase 21.5"" All-in-One Desktop, Intel ...",https://www.amazon.com/dp/B099P4T81H,B099P4T81H,HP,"{'value': 545, 'currency': '$'}",4.3,264.0,Electronics › Computers & Accessories › Comput...,https://m.media-amazon.com/images/I/81w3miL-DH...,Change your perspective on home PCs with the H...,[FLEXIBLE FAMILY FUN – Designed to live at the...,"[{'key': 'Standing screen display size', 'valu...",[],/HP-Chromebase-Processor-Touchscreen-22-aa0022...,True,[],[],Desktops,All-in-Ones


In [ ]:
# extract 'value' from the dict stored in 'price'
crawler_df['price'] = crawler_df['price'].apply(lambda x: x['value'] if isinstance(x, dict) else x)

# drop rows missing px
crawler_df = crawler_df[~crawler_df.price.isnull()]

crawler_df.head()

,title,url,asin,brand,price,stars,reviewsCount,thumbnailImage,description,features,attributes,variantAsins,reviewsLink,hasReviews,variantAttributes,manufacturerAttributes,category,sub-category
0,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",https://www.amazon.com/dp/B0BNM4WLBJ,B0BNM4WLBJ,sinbun,529.00,4.4,14.0,https://m.media-amazon.com/images/I/61oop4yoyF...,None,[【11th Intel Core i7 processor】11th i7 mini PC...,"[{'key': 'Processor', 'value': '4.7 GHz core_i...","[B0BNM4WLBJ, B0B8D2D7RN, B0BQ2BBRDV]",/i7-1165G7-Thunderbolt-Desktop-Computer-Comput...,True,"[{'key': 'Size', 'value': 'I7-1165G7 16G/512G'}]","[{'key': 'Thunderbolt 4.0 Type-C Port', 'value...",Desktops,Minis
1,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",https://www.amazon.com/dp/B0BQVM2XL6,B0BQVM2XL6,GEEKOM,599.99,4.7,54.0,https://m.media-amazon.com/images/I/51W6MBfVCT...,None,[[A Mini PC with An Upgraded Processor] GEEKOM...,"[{'key': 'Standing screen display size', 'valu...","[B0BQVM2XL6, B0BQWCQ7ZY]",/GEEKOM-Mini-IT11-i7-11390H-Computer/product-r...,True,"[{'key': 'Size', 'value': 'i711390H-32GB+1TB'}]",[],Desktops,Minis
3,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,https://www.amazon.com/dp/B099CHVFJJ,B099CHVFJJ,AVGPC,549.00,3.9,23.0,https://m.media-amazon.com/images/I/61LAnQI-iH...,Product images used on this page are for illus...,[AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD ...,"[{'key': 'Processor', 'value': '4.8 GHz ryzen_...","[B099CHVFJJ, B09X2JCX8Z, B0BGYRWRH7, B099CSM7L...",/AVGPC-Q-Box-Gaming-PC-Q-Box_7/product-reviews...,True,"[{'key': 'Style', 'value': 'Mini_B_5700G'}]",[],Desktops,Towers
4,Apple 2023 Mac Mini Desktop Computer M2 chip w...,https://www.amazon.com/dp/B0BSHGHGXR,B0BSHGHGXR,Apple,559.00,4.6,184.0,https://m.media-amazon.com/images/I/61La8PAa42...,None,[SUPERCHARGED BY M2 — Get more done faster wit...,[],"[B0BSHGHGXR, B0BSHFS6XX, B0BSHGHGXR, B0BSHGTPMV]",/Apple-Desktop-Computer-10%E2%80%91core-Ethern...,True,"[{'key': 'Style', 'value': 'Apple M2 Chip'}, {...",[],Desktops,Minis
6,"HP Chromebase 21.5"" All-in-One Desktop, Intel ...",https://www.amazon.com/dp/B099P4T81H,B099P4T81H,HP,545.00,4.3,264.0,https://m.media-amazon.com/images/I/81w3miL-DH...,Change your perspective on home PCs with the H...,[FLEXIBLE FAMILY FUN – Designed to live at the...,"[{'key': 'Standing screen display size', 'valu...",[],/HP-Chromebase-Processor-Touchscreen-22-aa0022...,True,[],[],Desktops,All-in-Ones


### We want both "relational" (i.e. desktop/laptop spec data) and unstructured (i.e. reviews) data for each product in our db
So we need to drop:  
* products from the `crawler` table that don't have any reviews
* products from the `reviews` table that we've removed from the `crawler` table due to data issues

In [ ]:
crawler_df = crawler_df[crawler_df['hasReviews'] == True]

In [ ]:
reviews_df = reviews_df[reviews_df.productAsin.isin(crawler_df.asin)]
len(reviews_df)

23231

In [ ]:
crawler_df.drop(columns=['reviewsLink', 'hasReviews'], inplace=True)
crawler_df.head()

,title,url,asin,brand,price,stars,reviewsCount,thumbnailImage,description,features,attributes,variantAsins,variantAttributes,manufacturerAttributes,category,sub-category
0,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",https://www.amazon.com/dp/B0BNM4WLBJ,B0BNM4WLBJ,sinbun,529.00,4.4,14.0,https://m.media-amazon.com/images/I/61oop4yoyF...,None,[【11th Intel Core i7 processor】11th i7 mini PC...,"[{'key': 'Processor', 'value': '4.7 GHz core_i...","[B0BNM4WLBJ, B0B8D2D7RN, B0BQ2BBRDV]","[{'key': 'Size', 'value': 'I7-1165G7 16G/512G'}]","[{'key': 'Thunderbolt 4.0 Type-C Port', 'value...",Desktops,Minis
1,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",https://www.amazon.com/dp/B0BQVM2XL6,B0BQVM2XL6,GEEKOM,599.99,4.7,54.0,https://m.media-amazon.com/images/I/51W6MBfVCT...,None,[[A Mini PC with An Upgraded Processor] GEEKOM...,"[{'key': 'Standing screen display size', 'valu...","[B0BQVM2XL6, B0BQWCQ7ZY]","[{'key': 'Size', 'value': 'i711390H-32GB+1TB'}]",[],Desktops,Minis
3,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,https://www.amazon.com/dp/B099CHVFJJ,B099CHVFJJ,AVGPC,549.00,3.9,23.0,https://m.media-amazon.com/images/I/61LAnQI-iH...,Product images used on this page are for illus...,[AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD ...,"[{'key': 'Processor', 'value': '4.8 GHz ryzen_...","[B099CHVFJJ, B09X2JCX8Z, B0BGYRWRH7, B099CSM7L...","[{'key': 'Style', 'value': 'Mini_B_5700G'}]",[],Desktops,Towers
4,Apple 2023 Mac Mini Desktop Computer M2 chip w...,https://www.amazon.com/dp/B0BSHGHGXR,B0BSHGHGXR,Apple,559.00,4.6,184.0,https://m.media-amazon.com/images/I/61La8PAa42...,None,[SUPERCHARGED BY M2 — Get more done faster wit...,[],"[B0BSHGHGXR, B0BSHFS6XX, B0BSHGHGXR, B0BSHGTPMV]","[{'key': 'Style', 'value': 'Apple M2 Chip'}, {...",[],Desktops,Minis
6,"HP Chromebase 21.5"" All-in-One Desktop, Intel ...",https://www.amazon.com/dp/B099P4T81H,B099P4T81H,HP,545.00,4.3,264.0,https://m.media-amazon.com/images/I/81w3miL-DH...,Change your perspective on home PCs with the H...,[FLEXIBLE FAMILY FUN – Designed to live at the...,"[{'key': 'Standing screen display size', 'valu...",[],[],[],Desktops,All-in-Ones


In [ ]:
print(len(crawler_df))
print(crawler_df.isnull().sum())

1661
title                       0
url                         0
asin                        0
brand                       0
price                       0
stars                       0
reviewsCount                0
thumbnailImage              0
description               579
features                    0
attributes                  0
variantAsins                0
variantAttributes           0
manufacturerAttributes      0
category                    0
sub-category                0
dtype: int64


In [ ]:
# create a new column in crawler_df that equals 1 if the product is a bestseller, 0 otherwise
crawler_df['bestseller'] = crawler_df['asin'].isin(bestsellers_df['asin']).astype(int)

In [ ]:
crawler_df.head()

,title,url,asin,brand,price,stars,reviewsCount,thumbnailImage,description,features,attributes,variantAsins,variantAttributes,manufacturerAttributes,category,sub-category,bestseller
0,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",https://www.amazon.com/dp/B0BNM4WLBJ,B0BNM4WLBJ,sinbun,529.00,4.4,14.0,https://m.media-amazon.com/images/I/61oop4yoyF...,None,[【11th Intel Core i7 processor】11th i7 mini PC...,"[{'key': 'Processor', 'value': '4.7 GHz core_i...","[B0BNM4WLBJ, B0B8D2D7RN, B0BQ2BBRDV]","[{'key': 'Size', 'value': 'I7-1165G7 16G/512G'}]","[{'key': 'Thunderbolt 4.0 Type-C Port', 'value...",Desktops,Minis,0
1,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",https://www.amazon.com/dp/B0BQVM2XL6,B0BQVM2XL6,GEEKOM,599.99,4.7,54.0,https://m.media-amazon.com/images/I/51W6MBfVCT...,None,[[A Mini PC with An Upgraded Processor] GEEKOM...,"[{'key': 'Standing screen display size', 'valu...","[B0BQVM2XL6, B0BQWCQ7ZY]","[{'key': 'Size', 'value': 'i711390H-32GB+1TB'}]",[],Desktops,Minis,0
3,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,https://www.amazon.com/dp/B099CHVFJJ,B099CHVFJJ,AVGPC,549.00,3.9,23.0,https://m.media-amazon.com/images/I/61LAnQI-iH...,Product images used on this page are for illus...,[AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD ...,"[{'key': 'Processor', 'value': '4.8 GHz ryzen_...","[B099CHVFJJ, B09X2JCX8Z, B0BGYRWRH7, B099CSM7L...","[{'key': 'Style', 'value': 'Mini_B_5700G'}]",[],Desktops,Towers,0
4,Apple 2023 Mac Mini Desktop Computer M2 chip w...,https://www.amazon.com/dp/B0BSHGHGXR,B0BSHGHGXR,Apple,559.00,4.6,184.0,https://m.media-amazon.com/images/I/61La8PAa42...,None,[SUPERCHARGED BY M2 — Get more done faster wit...,[],"[B0BSHGHGXR, B0BSHFS6XX, B0BSHGHGXR, B0BSHGTPMV]","[{'key': 'Style', 'value': 'Apple M2 Chip'}, {...",[],Desktops,Minis,1
6,"HP Chromebase 21.5"" All-in-One Desktop, Intel ...",https://www.amazon.com/dp/B099P4T81H,B099P4T81H,HP,545.00,4.3,264.0,https://m.media-amazon.com/images/I/81w3miL-DH...,Change your perspective on home PCs with the H...,[FLEXIBLE FAMILY FUN – Designed to live at the...,"[{'key': 'Standing screen display size', 'valu...",[],[],[],Desktops,All-in-Ones,0


## Make a column that combines all text data for semantic similarity search

In [ ]:
import string

def clean_string(text):                                                                                           
    return ''.join(char for char in text if char.isalnum() or char in string.punctuation + ' ') 

In [ ]:
# replace empty lists in the features col with an empty string
crawler_df['features'] = crawler_df['features'].apply(lambda x: '' if x == [] else x)

In [ ]:
crawler_df['features_temp'] = crawler_df['features'].apply(lambda x: '. '.join(x) if isinstance(x, list) else x).apply(clean_string)

In [ ]:
crawler_df['description'].fillna('', inplace=True)
crawler_df['description_temp'] = crawler_df['description'].apply(clean_string)

In [ ]:
crawler_df['seller_text'] = crawler_df['features_temp'] + '. ' + crawler_df['description_temp']
crawler_df.drop(columns=['description_temp', 'features_temp'], inplace=True)

crawler_df['title_text'] = crawler_df['title'].apply(clean_string)
crawler_df['all_product_text'] = crawler_df['seller_text'] + '. ' + crawler_df['title_text']

In [ ]:
crawler_df.head()

,title,url,asin,brand,price,stars,reviewsCount,thumbnailImage,description,features,attributes,variantAsins,variantAttributes,manufacturerAttributes,category,sub-category,bestseller,seller_text,title_text,all_product_text
0,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",https://www.amazon.com/dp/B0BNM4WLBJ,B0BNM4WLBJ,sinbun,529.00,4.4,14.0,https://m.media-amazon.com/images/I/61oop4yoyF...,,[【11th Intel Core i7 processor】11th i7 mini PC...,"[{'key': 'Processor', 'value': '4.7 GHz core_i...","[B0BNM4WLBJ, B0B8D2D7RN, B0BQ2BBRDV]","[{'key': 'Size', 'value': 'I7-1165G7 16G/512G'}]","[{'key': 'Thunderbolt 4.0 Type-C Port', 'value...",Desktops,Minis,0,11th Intel Core i7 processor11th i7 mini PC co...,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",11th Intel Core i7 processor11th i7 mini PC co...
1,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",https://www.amazon.com/dp/B0BQVM2XL6,B0BQVM2XL6,GEEKOM,599.99,4.7,54.0,https://m.media-amazon.com/images/I/51W6MBfVCT...,,[[A Mini PC with An Upgraded Processor] GEEKOM...,"[{'key': 'Standing screen display size', 'valu...","[B0BQVM2XL6, B0BQWCQ7ZY]","[{'key': 'Size', 'value': 'i711390H-32GB+1TB'}]",[],Desktops,Minis,0,[A Mini PC with An Upgraded Processor] GEEKOM ...,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",[A Mini PC with An Upgraded Processor] GEEKOM ...
3,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,https://www.amazon.com/dp/B099CHVFJJ,B099CHVFJJ,AVGPC,549.00,3.9,23.0,https://m.media-amazon.com/images/I/61LAnQI-iH...,Product images used on this page are for illus...,[AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD ...,"[{'key': 'Processor', 'value': '4.8 GHz ryzen_...","[B099CHVFJJ, B09X2JCX8Z, B0BGYRWRH7, B099CSM7L...","[{'key': 'Style', 'value': 'Mini_B_5700G'}]",[],Desktops,Towers,0,AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD R...,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD R...
4,Apple 2023 Mac Mini Desktop Computer M2 chip w...,https://www.amazon.com/dp/B0BSHGHGXR,B0BSHGHGXR,Apple,559.00,4.6,184.0,https://m.media-amazon.com/images/I/61La8PAa42...,,[SUPERCHARGED BY M2 — Get more done faster wit...,[],"[B0BSHGHGXR, B0BSHFS6XX, B0BSHGHGXR, B0BSHGTPMV]","[{'key': 'Style', 'value': 'Apple M2 Chip'}, {...",[],Desktops,Minis,1,SUPERCHARGED BY M2 Get more done faster with ...,Apple 2023 Mac Mini Desktop Computer M2 chip w...,SUPERCHARGED BY M2 Get more done faster with ...
6,"HP Chromebase 21.5"" All-in-One Desktop, Intel ...",https://www.amazon.com/dp/B099P4T81H,B099P4T81H,HP,545.00,4.3,264.0,https://m.media-amazon.com/images/I/81w3miL-DH...,Change your perspective on home PCs with the H...,[FLEXIBLE FAMILY FUN – Designed to live at the...,"[{'key': 'Standing screen display size', 'valu...",[],[],[],Desktops,All-in-Ones,0,FLEXIBLE FAMILY FUN Designed to live at the h...,"HP Chromebase 21.5"" All-in-One Desktop, Intel ...",FLEXIBLE FAMILY FUN Designed to live at the h...


In [ ]:
crawler_df.to_excel('cleaned_crawler_data.xlsx')

## Clean reviews data

In [ ]:
reviews_df.head()

,productAsin,ratingScore,reviewTitle,reviewUrl,reviewReaction,reviewedIn,date,country,countryCode,reviewDescription
0,B0BYJWRSC1,4,Painful Laptop,https://www.amazon.com/gp/customer-reviews/R6N...,,"Reviewed in the United States 🇺🇸 on May 6, 2023",2023-05-06,United States,🇺🇸,This is a fast and elegant looking Windows 11 ...
1,B0BYJWRSC1,5,Has all the features,https://www.amazon.com/gp/customer-reviews/R39...,,"Reviewed in the United States 🇺🇸 on April 20, ...",2023-04-20,United States,🇺🇸,This laptop is fast! 64 mb of ram really crank...
2,B0BYJWRSC1,5,Great Service,https://www.amazon.com/gp/customer-reviews/R3V...,,"Reviewed in the United States 🇺🇸 on April 26, ...",2023-04-26,United States,🇺🇸,Fast delivery and product exactly as described
3,B0BT6W36HL,5,Lots of performance packed into a thin and lig...,https://www.amazon.com/gp/customer-reviews/R2V...,,"Reviewed in the United States on May 6, 2023",2023-05-06,United States,None,"The Samsung 16"" Galaxy Book3 Pro Business Lapt..."
4,B0BT6W36HL,5,great AMOLED screen,https://www.amazon.com/gp/customer-reviews/R37...,5 people found this helpful,"Reviewed in the United States on April 3, 2023",2023-04-03,United States,None,I bought the Book3 Ultra version of this lapto...


In [ ]:
display(reviews_df.country.value_counts().to_frame())
reviews_df.loc[reviews_df['country'] == 'Germany', 'reviewDescription'].iloc[0]

,country
United States,23407
Mexico,289
Canada,105
Germany,36
Brazil,35
United Kingdom,22
Spain,14
France,13
Italy,12
Singapore,10


'Der HISTTON Mini PC wurde gut verpackt in einem Karton geliefert.Enthalten ist neben dem PC auch Zubehör:-Das Netzteil mit Anschlusskabel und Netzkabeln für EU (Stecker E+F) und einem für (vor allem) Großbritannien (Stecker Typ G)-2 Aufschraubbare WLAN-Antennen-Strom sowie Datenkabel für zusätzliches SATA-Laufwerk-Befestigungsmaterial für Laufwerk, Montage an Halterungen-Befestigungsplatte-Eine DVD mit Software (macht keinen seriösen Eindruck)-Eine englischsprachige AnleitungNicht mitgeliefert wurde ein Tischständer, der in der Beschreibung gezeigt wird.Der Computer hat ein sehr robust wirkendes und einwandfrei verarbeitete Gehäuse aus Metall (Aluminium) mit Kühlrippen. Das sollte die Wärme dieses passiv gekühlten Geräts gut ableiten.Der Computer ist offensichtlich eher für den Einsatz in Industrie, Produktion etc. vorgesehen und nicht für den normalen Schreibtisch-Einsatz -aber dennoch kann man ihn natürlich auch da nützen.Der Mini PC verfügt über eine riesige Anzahl an Anschlüssen:A

In [ ]:
reviews_df = reviews_df[reviews_df.country.isin(['United States', 'Canada', 'United Kingdom', 'Australia'])]

In [ ]:
reviews_df.head()

,productAsin,ratingScore,reviewTitle,reviewUrl,reviewReaction,reviewedIn,date,country,countryCode,reviewDescription
0,B0BYJWRSC1,4,Painful Laptop,https://www.amazon.com/gp/customer-reviews/R6N...,,"Reviewed in the United States 🇺🇸 on May 6, 2023",2023-05-06,United States,🇺🇸,This is a fast and elegant looking Windows 11 ...
1,B0BYJWRSC1,5,Has all the features,https://www.amazon.com/gp/customer-reviews/R39...,,"Reviewed in the United States 🇺🇸 on April 20, ...",2023-04-20,United States,🇺🇸,This laptop is fast! 64 mb of ram really crank...
2,B0BYJWRSC1,5,Great Service,https://www.amazon.com/gp/customer-reviews/R3V...,,"Reviewed in the United States 🇺🇸 on April 26, ...",2023-04-26,United States,🇺🇸,Fast delivery and product exactly as described
3,B0BT6W36HL,5,Lots of performance packed into a thin and lig...,https://www.amazon.com/gp/customer-reviews/R2V...,,"Reviewed in the United States on May 6, 2023",2023-05-06,United States,None,"The Samsung 16"" Galaxy Book3 Pro Business Lapt..."
4,B0BT6W36HL,5,great AMOLED screen,https://www.amazon.com/gp/customer-reviews/R37...,5 people found this helpful,"Reviewed in the United States on April 3, 2023",2023-04-03,United States,None,I bought the Book3 Ultra version of this lapto...


In [ ]:
reviews_df['reviewDescription'] = reviews_df['reviewDescription'].apply(clean_string)
reviews_df['reviewTitle'] = reviews_df['reviewTitle'].apply(clean_string)

reviews_df['all_review_text'] = reviews_df['reviewTitle'] + '. ' + reviews_df['reviewDescription']

reviews_df.head()

,productAsin,ratingScore,reviewTitle,reviewUrl,reviewReaction,reviewedIn,date,country,countryCode,reviewDescription,all_review_text
0,B0BYJWRSC1,4,Painful Laptop,https://www.amazon.com/gp/customer-reviews/R6N...,,"Reviewed in the United States 🇺🇸 on May 6, 2023",2023-05-06,United States,🇺🇸,This is a fast and elegant looking Windows 11 ...,Painful Laptop. This is a fast and elegant loo...
1,B0BYJWRSC1,5,Has all the features,https://www.amazon.com/gp/customer-reviews/R39...,,"Reviewed in the United States 🇺🇸 on April 20, ...",2023-04-20,United States,🇺🇸,This laptop is fast! 64 mb of ram really crank...,Has all the features. This laptop is fast! 64 ...
2,B0BYJWRSC1,5,Great Service,https://www.amazon.com/gp/customer-reviews/R3V...,,"Reviewed in the United States 🇺🇸 on April 26, ...",2023-04-26,United States,🇺🇸,Fast delivery and product exactly as described,Great Service. Fast delivery and product exact...
3,B0BT6W36HL,5,Lots of performance packed into a thin and lig...,https://www.amazon.com/gp/customer-reviews/R2V...,,"Reviewed in the United States on May 6, 2023",2023-05-06,United States,None,"The Samsung 16"" Galaxy Book3 Pro Business Lapt...",Lots of performance packed into a thin and lig...
4,B0BT6W36HL,5,great AMOLED screen,https://www.amazon.com/gp/customer-reviews/R37...,5 people found this helpful,"Reviewed in the United States on April 3, 2023",2023-04-03,United States,None,I bought the Book3 Ultra version of this lapto...,great AMOLED screen. I bought the Book3 Ultra ...


In [ ]:
# calculate a weighted average of the numPeopleFoundHelpful column by grouping on productAsin and taking the sum of numPeopleFoundHelpful divided by the sum of 1 / (1 + date - date.min())
reviews_df['numPeopleFoundHelpful'] = reviews_df['reviewReaction'].apply(lambda x: 1 if x == '' else 2 if x.startswith('One') else 1 + int(x.split(' ')[0].replace(',', '')))
reviews_df['totalHelpful'] = reviews_df.groupby('productAsin')['numPeopleFoundHelpful'].transform('sum')
reviews_df['wavgHelpfulness'] = reviews_df['numPeopleFoundHelpful'] / reviews_df['totalHelpful']
reviews_df.head()

,productAsin,ratingScore,reviewTitle,reviewUrl,reviewReaction,reviewedIn,date,country,countryCode,reviewDescription,all_review_text,numPeopleFoundHelpful,totalHelpful,wavgHelpfulness
0,B0BYJWRSC1,4,Painful Laptop,https://www.amazon.com/gp/customer-reviews/R6N...,,"Reviewed in the United States 🇺🇸 on May 6, 2023",2023-05-06,United States,🇺🇸,This is a fast and elegant looking Windows 11 ...,Painful Laptop. This is a fast and elegant loo...,1,3,0.333333
1,B0BYJWRSC1,5,Has all the features,https://www.amazon.com/gp/customer-reviews/R39...,,"Reviewed in the United States 🇺🇸 on April 20, ...",2023-04-20,United States,🇺🇸,This laptop is fast! 64 mb of ram really crank...,Has all the features. This laptop is fast! 64 ...,1,3,0.333333
2,B0BYJWRSC1,5,Great Service,https://www.amazon.com/gp/customer-reviews/R3V...,,"Reviewed in the United States 🇺🇸 on April 26, ...",2023-04-26,United States,🇺🇸,Fast delivery and product exactly as described,Great Service. Fast delivery and product exact...,1,3,0.333333
3,B0BT6W36HL,5,Lots of performance packed into a thin and lig...,https://www.amazon.com/gp/customer-reviews/R2V...,,"Reviewed in the United States on May 6, 2023",2023-05-06,United States,None,"The Samsung 16"" Galaxy Book3 Pro Business Lapt...",Lots of performance packed into a thin and lig...,1,7,0.142857
4,B0BT6W36HL,5,great AMOLED screen,https://www.amazon.com/gp/customer-reviews/R37...,5 people found this helpful,"Reviewed in the United States on April 3, 2023",2023-04-03,United States,None,I bought the Book3 Ultra version of this lapto...,great AMOLED screen. I bought the Book3 Ultra ...,6,7,0.857143


In [ ]:
reviews_df = reviews_df[['productAsin',
 'ratingScore',
 'reviewTitle',
 'reviewUrl',
 'reviewReaction',
 'date',
 'reviewDescription',
 'all_review_text',
 'wavgHelpfulness']]

,productAsin,ratingScore,reviewTitle,reviewUrl,reviewReaction,date,reviewDescription,all_review_text,wavgHelpfulness
0,B0BYJWRSC1,4,Painful Laptop,https://www.amazon.com/gp/customer-reviews/R6N...,,2023-05-06,This is a fast and elegant looking Windows 11 ...,Painful Laptop. This is a fast and elegant loo...,0.333333
1,B0BYJWRSC1,5,Has all the features,https://www.amazon.com/gp/customer-reviews/R39...,,2023-04-20,This laptop is fast! 64 mb of ram really crank...,Has all the features. This laptop is fast! 64 ...,0.333333
2,B0BYJWRSC1,5,Great Service,https://www.amazon.com/gp/customer-reviews/R3V...,,2023-04-26,Fast delivery and product exactly as described,Great Service. Fast delivery and product exact...,0.333333
3,B0BT6W36HL,5,Lots of performance packed into a thin and lig...,https://www.amazon.com/gp/customer-reviews/R2V...,,2023-05-06,"The Samsung 16"" Galaxy Book3 Pro Business Lapt...",Lots of performance packed into a thin and lig...,0.142857
4,B0BT6W36HL,5,great AMOLED screen,https://www.amazon.com/gp/customer-reviews/R37...,5 people found this helpful,2023-04-03,I bought the Book3 Ultra version of this lapto...,great AMOLED screen. I bought the Book3 Ultra ...,0.857143
...,...,...,...,...,...,...,...,...,...
23945,B08MMQH98H,5,Seller Stepped Up and Handled a Problem in a G...,https://www.amazon.com/gp/customer-reviews/R30...,2 people found this helpful,2023-03-05,"While it is a great little machine, before I b...",Seller Stepped Up and Handled a Problem in a G...,0.033333
23946,B08MMQH98H,5,Nice Refurb Unit,https://www.amazon.com/gp/customer-reviews/R4J...,,2023-02-18,I ordered are refurbished Levono Thinkcentre s...,Nice Refurb Unit. I ordered are refurbished Le...,0.011111
23947,B08MMQH98H,5,Looks brand new!,https://www.amazon.com/gp/customer-reviews/R3R...,4 people found this helpful,2023-03-03,After using it for one month i don't have any ...,Looks brand new!. After using it for one month...,0.055556
23948,B08MMQH98H,5,Great Business Machine!,https://www.amazon.com/gp/customer-reviews/RFS...,,2023-04-09,This machine served my customer's needs perfec...,Great Business Machine!. This machine served m...,0.011111


In [ ]:
product_data = pd.read_excel('cleaned_crawler_data.xlsx')
product_data.asin.nunique()

1661

In [ ]:
non_matchers = list(set(product_data.asin.unique()) - set(reviews_df.productAsin.unique())) + list(set(reviews_df.productAsin.unique()) - set(product_data.asin.unique()))
reviews_df = reviews_df[~reviews_df.productAsin.isin(non_matchers)]
product_data = product_data[~product_data.asin.isin(non_matchers)]
print(f'unique products in reviews_df: {reviews_df.productAsin.nunique()}')
print(f'unique products in product_df: {product_data.asin.nunique()}')  

product_data.to_excel('final_product_data.xlsx', index=False)
reviews_df.to_excel('final_reviews_data.xlsx', index=False)

unique products in reviews_df: 1660
unique products in product_df: 1660


## Flatten attributes

In [ ]:
def map_os(os_string):
    if os_string is None:
        return None
    os_string = os_string.lower()
    if "window" in os_string or "win" in os_string or "w11" in os_string or "w-11" in os_string or "w 11" in os_string:
        return "Windows"
    elif "mac" in os_string or "os x" in os_string or "ios" in os_string:
        return "macOS"
    elif "linux" in os_string or "ubuntu" in os_string:
        return "Linux"
    else:
        return os_string
    
import ast
import pandas as pd

# Assuming your original DataFrame is named product_data and the copy is product_df
product_df = product_data[['asin', 'attributes']].copy()
product_df['attributes'] = product_df['attributes'].apply(ast.literal_eval)

# Explode the list into individual rows
product_df = product_df.explode('attributes')
product_df = product_df[~product_df['attributes'].isnull()]

# Convert the dictionaries into two separate columns
product_df[['key', 'value']] = pd.DataFrame(product_df['attributes'].tolist(), index=product_df.index)

# Drop the 'attributes' column as it is no longer needed
product_df = product_df.drop(columns='attributes')

# Pivot the DataFrame so that each unique 'key' gets its own column, using asin as the index
product_df = product_df.pivot(index='asin', columns='key', values='value')

# Reset the index of the DataFrame
product_df = product_df.reset_index()

# Merge the pivoted DataFrame back to the original DataFrame
flat_product_df = pd.merge(product_data, product_df, on='asin', how='left')

flat_product_df[flat_product_df['Operating System'].apply(lambda x: isinstance(x, str))]


# drop if null in Brand, item model number, operating system, price, or processor
flat_product_df = flat_product_df.dropna(subset=['Brand', 'Item model number', 'Operating System', 'price', 'Processor'])
flat_product_df = flat_product_df[flat_product_df['Operating System'].apply(lambda x: isinstance(x, str))]
    
flat_product_df['Operating System'] = flat_product_df['Operating System'].apply(map_os)
flat_product_df = flat_product_df[['asin', 'Brand', 'Item model number', 'Operating System', 'price', 'RAM', 'Hard Drive', 'category', 'Series', 'Processor Brand', 'Processor', 'Chipset Brand', 'Graphics Coprocessor', 'variantAsins']]

# if `graphics coprocessor` contains `chipset brand` value, remove the matching string from `graphics coprocessor`
flat_product_df['Graphics Coprocessor'] = flat_product_df.apply(lambda row: row['Graphics Coprocessor'].replace(row['Chipset Brand'], '') if isinstance(row['Graphics Coprocessor'], str) and isinstance(row['Chipset Brand'], str) and row['Chipset Brand'].lower() in row['Graphics Coprocessor'].lower() else row['Graphics Coprocessor'], axis=1)
# strip leading and trailing whitespace
flat_product_df['Graphics Coprocessor'] = flat_product_df['Graphics Coprocessor'].apply(lambda x: x.strip() if isinstance(x, str) else x)
# set chipset brand to nan if graphics coprocessor is nan
flat_product_df['Chipset Brand'] = flat_product_df.apply(lambda row: np.nan if isinstance(row['Graphics Coprocessor'], float) and np.isnan(row['Graphics Coprocessor']) else row['Chipset Brand'], axis=1)

# drop instances where graphics coprocessor is nan but chipset brand is not nan
flat_product_df.loc[flat_product_df['Chipset Brand'].isnull() & flat_product_df['Graphics Coprocessor'].notnull()].index

flat_product_df = flat_product_df.drop(flat_product_df.loc[flat_product_df['Chipset Brand'].isnull() & flat_product_df['Graphics Coprocessor'].notnull()].index)

full_product_df = pd.merge(flat_product_df, product_data[['asin', 'bestseller', 'seller_text', 'title_text', 'url', 'stars', 'reviewsCount', 'thumbnailImage']], on='asin', how='left')
full_product_df = full_product_df[['asin', 'title_text', 'category', 'Series', 'Brand', 'Item model number', 'Operating System', 'price', 
                                   'RAM', 'Hard Drive', 'Processor Brand', 'Processor', 'Chipset Brand', 'Graphics Coprocessor', 
                                   'bestseller', 'seller_text', 'url', 'stars', 'reviewsCount', 'thumbnailImage', 'variantAsins']]
full_product_df.head(20)

,asin,title_text,category,Series,Brand,Item model number,Operating System,price,RAM,Hard Drive,Processor Brand,Processor,Chipset Brand,Graphics Coprocessor,bestseller,seller_text,url,stars,reviewsCount,thumbnailImage,variantAsins
0,B0BNM4WLBJ,"Mini PC, 11th Intel i7-1165G7(up to 4.7GHz) Th...",Desktops,BQM3 I7,Sinbun,BQM3 i7 16/512 GB,Windows,529.00,512 GB DDR4,16 GB SSD,Intel,4.7 GHz core_i7,NaN,NaN,0,11th Intel Core i7 processor11th i7 mini PC co...,https://www.amazon.com/dp/B0BNM4WLBJ,4.4,14,https://m.media-amazon.com/images/I/61oop4yoyF...,"['B0BNM4WLBJ', 'B0B8D2D7RN', 'B0BQ2BBRDV']"
1,B0BQVM2XL6,"GEEKOM Mini IT11 Mini PC, 1TB Intel i7-11390H ...",Desktops,Mini IT11,GEEKOM,Mini IT11,Windows,599.99,32 GB DDR4,1 TB,Intel,2400 MHz apple_ci7,NaN,NaN,0,[A Mini PC with An Upgraded Processor] GEEKOM ...,https://www.amazon.com/dp/B0BQVM2XL6,4.7,54,https://m.media-amazon.com/images/I/51W6MBfVCT...,"['B0BQVM2XL6', 'B0BQWCQ7ZY']"
2,B099CHVFJJ,AVGPC Q-Box Mini Series PC - AMD Ryzen 5 5600G...,Desktops,Q-Box,AVGPC,Q Box mini,Windows,549.00,16 GB DDR4,500 GB SSD,AMD,4.8 GHz ryzen_5,NaN,NaN,0,AMD Ryzen 5 5600G 3.9Ghz with Integrated AMD R...,https://www.amazon.com/dp/B099CHVFJJ,3.9,23,https://m.media-amazon.com/images/I/61LAnQI-iH...,"['B099CHVFJJ', 'B09X2JCX8Z', 'B0BGYRWRH7', 'B0..."
3,B099P4T81H,"HP Chromebase 21.5"" All-in-One Desktop, Intel ...",Desktops,22-aa0022,HP,22-aa0022,chrome os,545.00,4 GB DDR4,128 GB SSD,Intel,2.4 GHz pentium_gold_g5600,Intel,UHD Graphics 600,0,FLEXIBLE FAMILY FUN Designed to live at the h...,https://www.amazon.com/dp/B099P4T81H,4.3,264,https://m.media-amazon.com/images/I/81w3miL-DH...,[]
4,B09YVWMLBP,Dell 2022 Newest Optiplex 3090 Micro Form Fact...,Desktops,Optiplex,Dell,3090,Windows,624.13,16 GB DDR4,512 GB SSD,Intel,2.3 GHz core_i5,Intel,UHD Graphics,0,High Speed RAM And Enormous Space16GB high-ban...,https://www.amazon.com/dp/B09YVWMLBP,5.0,10,https://m.media-amazon.com/images/I/61TIHYXkb4...,"['B0B2VB5ZT1', 'B0B2V1BJYX', 'B09YVWMLBP', 'B0..."
5,B0BS2LCB1X,2018 Apple Mac Mini with 3.2GHz Intel Core i7 ...,Desktops,Apple Mac Mini,Apple,MRTT2LL/A,macOS,515.00,DDR4,128 GB SSD,Intel,3.2 GHz apple_ci7,Intel,UHD Graphics 630,0,"This pre-owned product is not Apple certified,...",https://www.amazon.com/dp/B0BS2LCB1X,4.6,29,https://m.media-amazon.com/images/I/61mujJvG+C...,[]
6,B0BWPKK7RN,Dell OptiPlex 7080 Micro Form Factor Mini Busi...,Desktops,OptiPlex,Dell,7080,Windows,599.00,32 GB DDR4,1 TB SSD,Intel,2.3 GHz core_i5,Intel,UHD Graphics,0,High Speed RAM And Enormous Space32GB high-ban...,https://www.amazon.com/dp/B0BWPKK7RN,4.1,13,https://m.media-amazon.com/images/I/51qO-k6MY1...,"['B0BWQ3F343', 'B0BX21XTPP', 'B0BWQ1PXL3', 'B0..."
7,B0B2KVQJM8,"Acer Aspire C24-1700-UA91 AIO Desktop | 23.8"" ...",Desktops,C24-1700-UA91,Acer,DQ.BJFAA.002,Windows,528.67,8 GB DDR4,512 GB SSD,Intel,4.4 GHz apple_ci3,NaN,NaN,0,SLIM. STYLISH. STUNNING: Save the clutter on y...,https://www.amazon.com/dp/B0B2KVQJM8,4.5,110,https://m.media-amazon.com/images/I/714AZcO8PL...,[]
8,B0BM8YLTH8,[Gaming PC] KAMRUI Mini PC AMD Ryzen 5 5600U U...,Desktops,AMR5-Ryzen 5 5600U,KAMRUI,AMR5,Windows,479.00,16 GB DDR4,"512 GB 512GB M.2 SSD Included, Support NVME/NG...",AMD,4.2 GHz ryzen_5,AMD,Radeon Vega 7,1,THE KEY TO VICTORYThe KAMRUI AMR5 mini gaming ...,https://www.amazon.com/dp/B0BM8YLTH8,4.5,166,https://m.media-amazon.com/images/I/61oUaIuI0A...,"['B0BM8YLTH8', 'B0BX82ZBMG']"
9,B0934RHLXD,"HP 600 G1 SFF Computer Desktop PC, Intel Core ...",Desktops,HP ProDesk 600 G1 SFF,HP,ProDesk,Windows,268.00,16 GB DDR3,128 GB HDD,Intel,3.4 GHz core_i7,Intel,Onboard HD Graphics,1,POWERFUL PROCESSOR Configured with Intels top...,https://www.amazon.com/dp/B0934RHLXD,3.9,262,https://m.media-amazon.com/images/I/81B91P9+6Y...,[]


In [ ]:
reviews_df = reviews_df[['productAsin',
 'reviewUrl',
 'ratingScore',
 'reviewTitle',
 'reviewDescription',
 'date',
 'all_review_text',
 'numPeopleFoundHelpful',
 'wavgHelpfulness']]

In [ ]:
reviews_df.head()

,productAsin,reviewUrl,ratingScore,reviewTitle,reviewDescription,date,all_review_text,numPeopleFoundHelpful,wavgHelpfulness
0,B0BYJWRSC1,https://www.amazon.com/gp/customer-reviews/R6N...,4,Painful Laptop,This is a fast and elegant looking Windows 11 ...,2023-05-06,Painful Laptop. This is a fast and elegant loo...,1,0.333333
1,B0BYJWRSC1,https://www.amazon.com/gp/customer-reviews/R39...,5,Has all the features,This laptop is fast! 64 mb of ram really crank...,2023-04-20,Has all the features. This laptop is fast! 64 ...,1,0.333333
2,B0BYJWRSC1,https://www.amazon.com/gp/customer-reviews/R3V...,5,Great Service,Fast delivery and product exactly as described,2023-04-26,Great Service. Fast delivery and product exact...,1,0.333333
3,B0BT6W36HL,https://www.amazon.com/gp/customer-reviews/R2V...,5,Lots of performance packed into a thin and lig...,"The Samsung 16"" Galaxy Book3 Pro Business Lapt...",2023-05-06,Lots of performance packed into a thin and lig...,1,0.142857
4,B0BT6W36HL,https://www.amazon.com/gp/customer-reviews/R37...,5,great AMOLED screen,I bought the Book3 Ultra version of this lapto...,2023-04-03,great AMOLED screen. I bought the Book3 Ultra ...,6,0.857143


In [ ]:
non_matchers = list(set(full_product_df.asin.unique()) - set(reviews_df.productAsin.unique())) + list(set(reviews_df.productAsin.unique()) - set(full_product_df.asin.unique()))
len(non_matchers)

319

In [ ]:
non_matchers = list(set(full_product_df.asin.unique()) - set(reviews_df.productAsin.unique())) + list(set(reviews_df.productAsin.unique()) - set(full_product_df.asin.unique()))
reviews_df = reviews_df[~reviews_df.productAsin.isin(non_matchers)]
full_product_df = full_product_df[~full_product_df.asin.isin(non_matchers)]
print(f'unique products in reviews_df: {reviews_df.productAsin.nunique()}')
print(f'unique products in product_df: {full_product_df.asin.nunique()}')  

unique products in reviews_df: 1341
unique products in product_df: 1341


In [ ]:
# drop all rows containing null values
reviews_df = reviews_df.dropna()
full_product_df = full_product_df.dropna()
full_product_df.to_excel('./amazon_data/full_product_data.xlsx', index=False)
reviews_df.to_excel('./amazon_data/reviews_df.xlsx', index=False)